In [1]:
import pandas as pd # Trabajos con datasets
import numpy as np # Funciones matemáticas
import matplotlib.pyplot as plt #Gráficos sencillos

### Analisis Exploratorio

7 etapas para un análisis exploratorio
    
1. Clarificar la pregunta o preguntas a responder
2. Tener una idea general del dataset o datasets
3. Verificar los tipos de datos que tenemos
4. Elegir estadísticas descriptivas para los datos
5. Visualizar datos
6. Analizar posibles interacciones entre variables
7. Extraer algunas conclusiones y eventualmente dar líneas directoras para un análisis más profundo

#### 1. Discutir con los requerimientos del análisis solicitado, para formular preguntas a responder con el dataset

#### 2. Mirar en detalle el dataset: ¿Se debe realizar etapas de limpieza, variables dummy u otros? ¿Se puede trabajar con él? 

In [2]:
Cereales = pd.read_csv('cereals/Cereal_data.txt', sep=" ", header=None,index_col=None)
Cereales

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,100%_Bran,N,C,70,4,1,130,10.0,5.0,6,3,280,25,1.0,0.33
1,100%_Natural_Bran,Q,C,120,3,5,15,2.0,8.0,8,3,135,0,1.0,-1.00
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,3,320,25,1.0,0.33
3,All-Bran_with_Extra_Fiber,K,C,50,4,0,140,14.0,8.0,0,3,330,25,1.0,0.50
4,Almond_Delight,R,C,110,2,2,200,1.0,14.0,8,3,-1,25,1.0,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,G,C,110,2,1,250,0.0,21.0,3,3,60,25,1.0,0.75
73,Trix,G,C,110,1,1,140,0.0,13.0,12,2,25,25,1.0,1.00
74,Wheat_Chex,R,C,100,3,1,230,3.0,17.0,3,1,115,25,1.0,0.67
75,Wheaties,G,C,100,3,1,200,3.0,17.0,3,1,110,25,1.0,1.00


In [3]:
D3=pd.read_excel('cereals/Cereal data columns.xlsx',header=None,index_col=0)
D3=D3.transpose()
D3['Cups']=[] #Falta ese índice en el xlsx según la lectura de la información de la base de datos
Cereales.columns=D3.columns #Copiando las columnas obtenidas en la base de datos anterior
Cereales

,Cereal,Manufacturer,Type,Calories,Protein,Fat,Sodium,Dietary Fiber,Complex Carbohydrates,Sugar,Display Shelf,Potassium,Vitamins & Minerals,Weight,Cups
0,100%_Bran,N,C,70,4,1,130,10.0,5.0,6,3,280,25,1.0,0.33
1,100%_Natural_Bran,Q,C,120,3,5,15,2.0,8.0,8,3,135,0,1.0,-1.00
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,3,320,25,1.0,0.33
3,All-Bran_with_Extra_Fiber,K,C,50,4,0,140,14.0,8.0,0,3,330,25,1.0,0.50
4,Almond_Delight,R,C,110,2,2,200,1.0,14.0,8,3,-1,25,1.0,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,G,C,110,2,1,250,0.0,21.0,3,3,60,25,1.0,0.75
73,Trix,G,C,110,1,1,140,0.0,13.0,12,2,25,25,1.0,1.00
74,Wheat_Chex,R,C,100,3,1,230,3.0,17.0,3,1,115,25,1.0,0.67
75,Wheaties,G,C,100,3,1,200,3.0,17.0,3,1,110,25,1.0,1.00


In [9]:
Cereales.shape

(77, 15)

In [4]:
Cereales.columns

Index(['Cereal', 'Manufacturer', 'Type', 'Calories', 'Protein', 'Fat',
       'Sodium', 'Dietary Fiber', 'Complex Carbohydrates', 'Sugar ',
       'Display Shelf', 'Potassium', 'Vitamins & Minerals', 'Weight', 'Cups'],
      dtype='object', name=0)

En la documentación de la base de datos, indican que las casillas con información "-1" corresponden a datos nulos. Si bien hay una opción de solicitar estos datos por correo, la base de datos es muy antigua (de los 1990), por lo que para tener información inmediata y coherente (poner 'Desconocido' o 0 en las numéricas afectaría la información), se recurre al relleno de datos con la media de cada columna con al menos un dato considerado como faltante

In [5]:
def searchNulls(dataframe,columns):
    for colu in columns:
        getter=dataframe[dataframe[colu]==-1]
        counter=getter[colu].count() #Obtener y contar las filas con datos faltantes por columna
        if counter>0:
            #Si hay datos faltantes, llenar con la media en el formato numérico según la columna
            if dataframe[colu].dtype=='float64':
                meanColu=round(dataframe[colu].mean(),2)
            else:
                meanColu=round(dataframe[colu].mean())
            dataframe.loc[(dataframe[colu]==-1),colu]=meanColu
    return dataframe

In [6]:
CerealMejorado=searchNulls(Cereales,Cereales.columns) #Borrando los -1 (nulos) con medias
CerealMejorado

,Cereal,Manufacturer,Type,Calories,Protein,Fat,Sodium,Dietary Fiber,Complex Carbohydrates,Sugar,Display Shelf,Potassium,Vitamins & Minerals,Weight,Cups
0,100%_Bran,N,C,70,4,1,130,10.0,5.0,6,3,280,25,1.0,0.33
1,100%_Natural_Bran,Q,C,120,3,5,15,2.0,8.0,8,3,135,0,1.0,0.59
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,3,320,25,1.0,0.33
3,All-Bran_with_Extra_Fiber,K,C,50,4,0,140,14.0,8.0,0,3,330,25,1.0,0.50
4,Almond_Delight,R,C,110,2,2,200,1.0,14.0,8,3,96,25,1.0,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,G,C,110,2,1,250,0.0,21.0,3,3,60,25,1.0,0.75
73,Trix,G,C,110,1,1,140,0.0,13.0,12,2,25,25,1.0,1.00
74,Wheat_Chex,R,C,100,3,1,230,3.0,17.0,3,1,115,25,1.0,0.67
75,Wheaties,G,C,100,3,1,200,3.0,17.0,3,1,110,25,1.0,1.00


In [11]:
CerealMejorado.describe()

,Calories,Protein,Fat,Sodium,Dietary Fiber,Complex Carbohydrates,Sugar,Display Shelf,Potassium,Vitamins & Minerals,Weight,Cups
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,106.883117,2.545455,1.012987,159.675325,2.151948,14.800000,7.025974,2.207792,98.597403,28.246753,1.029091,0.773117
std,19.484119,1.094790,1.006473,83.832295,2.383364,3.881603,4.349755,0.832524,69.479315,22.342523,0.150614,0.233149
min,50.000000,1.000000,0.000000,0.000000,0.000000,5.000000,0.000000,1.000000,15.000000,0.000000,0.500000,0.250000
25%,100.000000,2.000000,0.000000,130.000000,1.000000,12.000000,3.000000,1.000000,45.000000,25.000000,1.000000,0.590000
50%,110.000000,3.000000,1.000000,180.000000,2.000000,14.600000,7.000000,2.000000,90.000000,25.000000,1.000000,0.750000
75%,110.000000,3.000000,2.000000,210.000000,3.000000,17.000000,11.000000,3.000000,120.000000,25.000000,1.000000,1.000000
max,160.000000,6.000000,5.000000,320.000000,14.000000,23.000000,15.000000,3.000000,330.000000,100.000000,1.500000,1.500000


In [7]:
CerealMejorado.Manufacturer.unique()

array(['N', 'Q', 'K', 'R', 'G', 'P', 'A'], dtype=object)

- A: American Home Food Products;
- G: General Mills;
- K: Kelloggs;
- N: Nabisco;
- P: Post;
- Q: Quaker Oats;
- R: Ralston Purina

In [8]:
CerealMejorado.Type.unique()

array(['C', 'H'], dtype=object)

- C: Cold
- H: Hot

#### 3. Una vez el dataset está limpio y se tienen los objetivos, verificar las variables, longitud del dataset, tipos de variables (categóricas o cuantitativas), realizar variables dummy si es necesario y que las variables cuantitativas estén en string

#### 4. Realizar estadísticas con las variables

#### 6. Realizar gráficas para ver las relaciones entre las variables (plot, nubes de puntos, etc)

#### 7. En función de lo visto anteriormente y con las preguntas en mente, sacar conclusiones y recomendaciones